# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-kk46am22
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-kk46am22
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit f5a7d455546a88cfdfb26e781d5bd6447e8243eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 2.1 MB/s eta 0:00:00
    

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [3]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [4]:
model = HookedTransformer.from_pretrained("gpt2-xl")

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-xl into HookedTransformer


# Fibonacci

In [5]:
example_prompt = "0, 1, 1, 2, 3, 5,"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ',', ' 1', ',', ' 1', ',', ' 2', ',', ' 3', ',', ' 5', ',']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 1        Logit: 19.12 Prob: 25.16% Token: | 8|

Top 0th token. Logit: 20.00 Prob: 60.66% Token: | 7|
Top 1th token. Logit: 19.12 Prob: 25.16% Token: | 8|
Top 2th token. Logit: 17.55 Prob:  5.24% Token: | 6|
Top 3th token. Logit: 17.38 Prob:  4.40% Token: | 9|
Top 4th token. Logit: 16.12 Prob:  1.25% Token: | 10|
Top 5th token. Logit: 15.16 Prob:  0.48% Token: | ...|
Top 6th token. Logit: 14.50 Prob:  0.25% Token: | …|
Top 7th token. Logit: 14.43 Prob:  0.23% Token: | X|
Top 8th token. Logit: 14.06 Prob:  0.16% Token: | 4|
Top 9th token. Logit: 13.96 Prob:  0.14% Token: |
|


Ranks of the answer tokens: [(' 8', 1)]

In [6]:
example_prompt = "0 1 1 2 3 5"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 1', ' 1', ' 2', ' 3', ' 5']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 2        Logit: 15.11 Prob:  8.13% Token: | 8|

Top 0th token. Logit: 16.67 Prob: 38.65% Token: | 6|
Top 1th token. Logit: 15.62 Prob: 13.61% Token: | 7|
Top 2th token. Logit: 15.11 Prob:  8.13% Token: | 8|
Top 3th token. Logit: 15.06 Prob:  7.74% Token: | 10|
Top 4th token. Logit: 14.74 Prob:  5.64% Token: | 9|
Top 5th token. Logit: 14.52 Prob:  4.51% Token: |
|
Top 6th token. Logit: 13.96 Prob:  2.59% Token: | 0|
Top 7th token. Logit: 13.69 Prob:  1.97% Token: | 5|
Top 8th token. Logit: 13.45 Prob:  1.55% Token: | 1|
Top 9th token. Logit: 13.23 Prob:  1.24% Token: | 11|


Ranks of the answer tokens: [(' 8', 2)]

In [7]:
example_prompt = "0 1 1 2 3 5 8 13 21"
example_answer = " 34"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 1', ' 1', ' 2', ' 3', ' 5', ' 8', ' 13', ' 21']
Tokenized answer: [' 34']


Performance on answer token:
Rank: 13       Logit: 14.42 Prob:  1.89% Token: | 34|

Top 0th token. Logit: 16.78 Prob: 20.05% Token: | 27|
Top 1th token. Logit: 16.40 Prob: 13.71% Token: | 25|
Top 2th token. Logit: 15.90 Prob:  8.29% Token: | 24|
Top 3th token. Logit: 15.70 Prob:  6.79% Token: | 28|
Top 4th token. Logit: 15.45 Prob:  5.30% Token: | ||
Top 5th token. Logit: 15.43 Prob:  5.17% Token: | 29|
Top 6th token. Logit: 15.39 Prob:  4.98% Token: |
|
Top 7th token. Logit: 15.27 Prob:  4.43% Token: | 26|
Top 8th token. Logit: 14.75 Prob:  2.63% Token: | 31|
Top 9th token. Logit: 14.65 Prob:  2.39% Token: | 33|


Ranks of the answer tokens: [(' 34', 13)]

# 2 4 6 8

In [8]:
example_prompt = "2 4 6 8 10 12"
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12']
Tokenized answer: [' 14']


Performance on answer token:
Rank: 0        Logit: 15.77 Prob: 41.64% Token: | 14|

Top 0th token. Logit: 15.77 Prob: 41.64% Token: | 14|
Top 1th token. Logit: 14.78 Prob: 15.51% Token: | 16|
Top 2th token. Logit: 14.09 Prob:  7.80% Token: |
|
Top 3th token. Logit: 14.05 Prob:  7.51% Token: | 15|
Top 4th token. Logit: 12.81 Prob:  2.16% Token: | 13|
Top 5th token. Logit: 12.66 Prob:  1.87% Token: | 20|
Top 6th token. Logit: 12.64 Prob:  1.83% Token: | 4|
Top 7th token. Logit: 12.43 Prob:  1.48% Token: | 18|
Top 8th token. Logit: 12.39 Prob:  1.43% Token: | 2|
Top 9th token. Logit: 12.39 Prob:  1.42% Token: | 1|


Ranks of the answer tokens: [(' 14', 0)]

In [9]:
example_prompt = "2 4 8 16 32 64"
example_answer = " 128"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 8', ' 16', ' 32', ' 64']
Tokenized answer: [' 128']


Performance on answer token:
Rank: 0        Logit: 15.70 Prob: 32.40% Token: | 128|

Top 0th token. Logit: 15.70 Prob: 32.40% Token: | 128|
Top 1th token. Logit: 14.81 Prob: 13.43% Token: |
|
Top 2th token. Logit: 13.70 Prob:  4.42% Token: | 80|
Top 3th token. Logit: 13.61 Prob:  4.05% Token: | 100|
Top 4th token. Logit: 13.49 Prob:  3.57% Token: |

|
Top 5th token. Logit: 13.11 Prob:  2.45% Token: | 96|
Top 6th token. Logit: 13.05 Prob:  2.31% Token: | 120|
Top 7th token. Logit: 12.65 Prob:  1.54% Token: | ||
Top 8th token. Logit: 12.14 Prob:  0.93% Token: | 72|
Top 9th token. Logit: 11.98 Prob:  0.79% Token: | 256|


Ranks of the answer tokens: [(' 128', 0)]

In [10]:
example_prompt = "4 8 16 32 64"
example_answer = " 128"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '4', ' 8', ' 16', ' 32', ' 64']
Tokenized answer: [' 128']


Performance on answer token:
Rank: 7        Logit: 13.34 Prob:  1.66% Token: | 128|

Top 0th token. Logit: 16.43 Prob: 36.51% Token: |%|
Top 1th token. Logit: 15.10 Prob:  9.65% Token: |
|
Top 2th token. Logit: 14.71 Prob:  6.53% Token: | per|
Top 3th token. Logit: 14.50 Prob:  5.27% Token: | %|
Top 4th token. Logit: 14.46 Prob:  5.06% Token: | 100|
Top 5th token. Logit: 14.14 Prob:  3.69% Token: | 80|
Top 6th token. Logit: 13.45 Prob:  1.84% Token: | 72|
Top 7th token. Logit: 13.34 Prob:  1.66% Token: | 128|
Top 8th token. Logit: 13.24 Prob:  1.49% Token: | 96|
Top 9th token. Logit: 13.18 Prob:  1.41% Token: | percent|


Ranks of the answer tokens: [(' 128', 7)]

# arithm plus 1

In [11]:
example_prompt = "1 + 1 ="
example_answer = " 2"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' +', ' 1', ' =']
Tokenized answer: [' 2']


Performance on answer token:
Rank: 0        Logit: 15.48 Prob: 35.73% Token: | 2|

Top 0th token. Logit: 15.48 Prob: 35.73% Token: | 2|
Top 1th token. Logit: 15.08 Prob: 23.97% Token: | 3|
Top 2th token. Logit: 13.57 Prob:  5.29% Token: | ?|
Top 3th token. Logit: 13.42 Prob:  4.56% Token: |
|
Top 4th token. Logit: 13.33 Prob:  4.17% Token: | 4|
Top 5th token. Logit: 13.02 Prob:  3.06% Token: | 5|
Top 6th token. Logit: 12.80 Prob:  2.45% Token: | 1|
Top 7th token. Logit: 12.39 Prob:  1.62% Token: | 0|
Top 8th token. Logit: 11.96 Prob:  1.06% Token: |?|
Top 9th token. Logit: 11.93 Prob:  1.03% Token: | 6|


Ranks of the answer tokens: [(' 2', 0)]

In [12]:
example_prompt = "5 + 1 ="
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '5', ' +', ' 1', ' =']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 4        Logit: 14.16 Prob:  8.56% Token: | 6|

Top 0th token. Logit: 14.55 Prob: 12.69% Token: | 7|
Top 1th token. Logit: 14.47 Prob: 11.75% Token: | 9|
Top 2th token. Logit: 14.34 Prob: 10.33% Token: |
|
Top 3th token. Logit: 14.20 Prob:  8.98% Token: | 10|
Top 4th token. Logit: 14.16 Prob:  8.56% Token: | 6|
Top 5th token. Logit: 13.96 Prob:  7.02% Token: | ?|
Top 6th token. Logit: 13.84 Prob:  6.21% Token: | 8|
Top 7th token. Logit: 13.49 Prob:  4.38% Token: | 5|
Top 8th token. Logit: 12.95 Prob:  2.56% Token: |?|
Top 9th token. Logit: 12.89 Prob:  2.41% Token: | 11|


Ranks of the answer tokens: [(' 6', 4)]

In [13]:
example_prompt = "10 + 1 ="
example_answer = " 11"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '10', ' +', ' 1', ' =']
Tokenized answer: [' 11']


Performance on answer token:
Rank: 1        Logit: 14.03 Prob: 10.22% Token: | 11|

Top 0th token. Logit: 14.26 Prob: 12.90% Token: | ?|
Top 1th token. Logit: 14.03 Prob: 10.22% Token: | 11|
Top 2th token. Logit: 13.90 Prob:  8.99% Token: |
|
Top 3th token. Logit: 13.58 Prob:  6.51% Token: | 14|
Top 4th token. Logit: 13.34 Prob:  5.13% Token: | 13|
Top 5th token. Logit: 13.32 Prob:  5.03% Token: | 10|
Top 6th token. Logit: 13.15 Prob:  4.26% Token: | 12|
Top 7th token. Logit: 13.09 Prob:  4.01% Token: | 15|
Top 8th token. Logit: 12.86 Prob:  3.19% Token: | 9|
Top 9th token. Logit: 12.75 Prob:  2.84% Token: | 20|


Ranks of the answer tokens: [(' 11', 1)]

In [14]:
example_prompt = "1 + 1 = 2. 10 + 1 = 11. 3 + 1 = 4. 5 + 1 ="
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' +', ' 1', ' =', ' 2', '.', ' 10', ' +', ' 1', ' =', ' 11', '.', ' 3', ' +', ' 1', ' =', ' 4', '.', ' 5', ' +', ' 1', ' =']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 20.19 Prob: 71.07% Token: | 6|

Top 0th token. Logit: 20.19 Prob: 71.07% Token: | 6|
Top 1th token. Logit: 18.66 Prob: 15.51% Token: | 7|
Top 2th token. Logit: 17.48 Prob:  4.77% Token: | 8|
Top 3th token. Logit: 17.08 Prob:  3.17% Token: | 9|
Top 4th token. Logit: 16.44 Prob:  1.68% Token: | 10|
Top 5th token. Logit: 16.10 Prob:  1.19% Token: | 5|
Top 6th token. Logit: 15.03 Prob:  0.41% Token: | 3|
Top 7th token. Logit: 14.96 Prob:  0.38% Token: | 11|
Top 8th token. Logit: 14.92 Prob:  0.37% Token: | 2|
Top 9th token. Logit: 14.58 Prob:  0.26% Token: | 12|


Ranks of the answer tokens: [(' 6', 0)]

In [15]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 5 table"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 5', ' table']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 17.92 Prob: 91.36% Token: | 6|

Top 0th token. Logit: 17.92 Prob: 91.36% Token: | 6|
Top 1th token. Logit: 14.41 Prob:  2.74% Token: |
|
Top 2th token. Logit: 13.60 Prob:  1.22% Token: | 5|
Top 3th token. Logit: 12.71 Prob:  0.50% Token: | 7|
Top 4th token. Logit: 12.33 Prob:  0.34% Token: | 1|
Top 5th token. Logit: 11.84 Prob:  0.21% Token: | 2|
Top 6th token. Logit: 11.60 Prob:  0.16% Token: | 11|
Top 7th token. Logit: 11.52 Prob:  0.15% Token: | (|
Top 8th token. Logit: 11.46 Prob:  0.14% Token: | 3|
Top 9th token. Logit: 11.31 Prob:  0.12% Token: |

|


Ranks of the answer tokens: [(' 6', 0)]

In [16]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 15 table"
example_answer = " 16"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 15', ' table']
Tokenized answer: [' 16']


Performance on answer token:
Rank: 16       Logit: 10.97 Prob:  0.20% Token: | 16|

Top 0th token. Logit: 16.70 Prob: 61.52% Token: | 5|
Top 1th token. Logit: 15.01 Prob: 11.35% Token: | 6|
Top 2th token. Logit: 14.69 Prob:  8.23% Token: |
|
Top 3th token. Logit: 14.04 Prob:  4.27% Token: | 12|
Top 4th token. Logit: 12.69 Prob:  1.11% Token: | 15|
Top 5th token. Logit: 12.52 Prob:  0.93% Token: | 2|
Top 6th token. Logit: 12.33 Prob:  0.78% Token: | 1|
Top 7th token. Logit: 12.33 Prob:  0.77% Token: | 8|
Top 8th token. Logit: 12.24 Prob:  0.71% Token: | 4|
Top 9th token. Logit: 12.02 Prob:  0.57% Token: | 10|


Ranks of the answer tokens: [(' 16', 16)]

In [17]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 101 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 101', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 2        Logit: 14.39 Prob: 11.67% Token: | 102|

Top 0th token. Logit: 14.72 Prob: 16.21% Token: | 5|
Top 1th token. Logit: 14.62 Prob: 14.59% Token: | 12|
Top 2th token. Logit: 14.39 Prob: 11.67% Token: | 102|
Top 3th token. Logit: 14.28 Prob: 10.41% Token: |
|
Top 4th token. Logit: 14.26 Prob: 10.15% Token: | 2|
Top 5th token. Logit: 13.50 Prob:  4.78% Token: | 6|
Top 6th token. Logit: 13.34 Prob:  4.08% Token: | 10|
Top 7th token. Logit: 13.06 Prob:  3.07% Token: | 101|
Top 8th token. Logit: 12.65 Prob:  2.03% Token: | 8|
Top 9th token. Logit: 12.56 Prob:  1.87% Token: | 4|


Ranks of the answer tokens: [(' 102', 2)]

# in-context greater-than

In [18]:
example_prompt = "1 table 4. 10 table 100. 3 table 6. 101 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 4', '.', ' 10', ' table', ' 100', '.', ' 3', ' table', ' 6', '.', ' 101', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 19       Logit: 12.40 Prob:  1.31% Token: | 102|

Top 0th token. Logit: 14.21 Prob:  8.00% Token: |
|
Top 1th token. Logit: 14.07 Prob:  6.95% Token: | 200|
Top 2th token. Logit: 14.06 Prob:  6.90% Token: | 100|
Top 3th token. Logit: 14.01 Prob:  6.54% Token: | 10|
Top 4th token. Logit: 13.80 Prob:  5.35% Token: | 1000|
Top 5th token. Logit: 13.75 Prob:  5.06% Token: | 6|
Top 6th token. Logit: 13.65 Prob:  4.58% Token: | 3|
Top 7th token. Logit: 13.57 Prob:  4.24% Token: | 2|
Top 8th token. Logit: 13.42 Prob:  3.62% Token: | 1|
Top 9th token. Logit: 13.39 Prob:  3.53% Token: | 101|


Ranks of the answer tokens: [(' 102', 19)]

In [19]:
example_prompt = "1 table 4. 10 table 100. 3 table 6. 1 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 4', '.', ' 10', ' table', ' 100', '.', ' 3', ' table', ' 6', '.', ' 1', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 113      Logit:  7.97 Prob:  0.02% Token: | 102|

Top 0th token. Logit: 14.06 Prob: 10.53% Token: | 3|
Top 1th token. Logit: 13.92 Prob:  9.21% Token: | 6|
Top 2th token. Logit: 13.82 Prob:  8.35% Token: | 4|
Top 3th token. Logit: 13.77 Prob:  7.89% Token: | 5|
Top 4th token. Logit: 13.76 Prob:  7.81% Token: | 10|
Top 5th token. Logit: 13.65 Prob:  6.99% Token: | 1|
Top 6th token. Logit: 13.64 Prob:  6.93% Token: | 2|
Top 7th token. Logit: 13.34 Prob:  5.13% Token: | 8|
Top 8th token. Logit: 13.08 Prob:  3.97% Token: | 7|
Top 9th token. Logit: 13.04 Prob:  3.82% Token: |
|


Ranks of the answer tokens: [(' 102', 113)]

In [20]:
example_prompt = "1 table 4. 10 table 100. 3 table 6. 4 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 4', '.', ' 10', ' table', ' 100', '.', ' 3', ' table', ' 6', '.', ' 4', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 81       Logit:  8.79 Prob:  0.03% Token: | 102|

Top 0th token. Logit: 14.74 Prob: 10.94% Token: | 8|
Top 1th token. Logit: 14.68 Prob: 10.35% Token: | 10|
Top 2th token. Logit: 14.51 Prob:  8.74% Token: | 4|
Top 3th token. Logit: 14.48 Prob:  8.46% Token: | 3|
Top 4th token. Logit: 14.41 Prob:  7.87% Token: | 5|
Top 5th token. Logit: 14.28 Prob:  6.92% Token: | 6|
Top 6th token. Logit: 14.13 Prob:  5.94% Token: | 1|
Top 7th token. Logit: 14.11 Prob:  5.87% Token: | 2|
Top 8th token. Logit: 14.11 Prob:  5.82% Token: | 7|
Top 9th token. Logit: 13.62 Prob:  3.57% Token: | 100|


Ranks of the answer tokens: [(' 102', 81)]